Imports

In [1]:
import gurobipy as gb
import pandas as pd
import numpy as np
import ast


Column Generation from Pre-processed Data

In [2]:
def indices_to_list(indices, length):
    return [1 if i in indices else 0 for i in range(length+1)][1:]

In [3]:
pre_data = pd.read_csv('combinations.tsv',sep='\t')
pre_data['Chairs'] = pre_data['Chairs'].apply(ast.literal_eval)
pre_data['Members'] = pre_data['Members'].apply(ast.literal_eval)
pre_data['Chairs'] = pre_data['Chairs'].apply(lambda x: indices_to_list(x,10))
pre_data['Members'] = pre_data['Members'].apply(lambda x: indices_to_list(x,30))
pre_data['a'] = pre_data['Members'] + pre_data['Chairs']
pre_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16344393 entries, 0 to 16344392
Data columns (total 5 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Day      int64 
 1   Time     int64 
 2   Members  object
 3   Chairs   object
 4   a        object
dtypes: int64(2), object(3)
memory usage: 623.5+ MB


In [4]:
data = pre_data[['Day','Time','a']]

In [5]:
data['h'] = data.groupby(['Day', 'Time'])['a'].cumcount() + 1


/var/folders/tf/frrjsdzx0fj9r1dvbn0zqj5m0000gn/T/ipykernel_93869/107800561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['h'] = data.groupby(['Day', 'Time'])['a'].cumcount() + 1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16344393 entries, 0 to 16344392
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Day     int64 
 1   Time    int64 
 2   a       object
 3   h       int64 
dtypes: int64(3), object(1)
memory usage: 498.8+ MB


The meat and potatoes of it all...

In [7]:
def maximize_meetings(data,filename='sol'):
    model = gb.Model("Meeting Scheduling")

    # Decision variables
    meetings = {}

    # Indices
    times = range(1,35)
    days = range(1,6)
    persons = range(0,40)

    #Generated columns hidden with dict{ day : dict{ time : [col_1,col_2,...col_h]}}
    cols = {
        1 : {}, #here would contain times 1-34, followed by a list of list/vectors/cols
        2 : {},
        3 : {},
        4 : {},
        5 : {}
    }
    for d in days: # days 1-5
        for t in times: # time slots 1-34
            cols[d][t] = data[(data['Day'] == d) & (data['Time'] == t)]['a'].tolist()
            for h in range(len(cols[d][t])): #iterate over possible columns
                # Decision variables
                meetings[(t, d, h)] = model.addVar(vtype=gb.GRB.BINARY, name=f"x_{t}_{d}_{h}")

    for d in days: # days 1-5
        for t in times: # time slots 1-34
            num_cols = len(cols[d][t])

            # Constraint 1: At most 4 meetings per time slot and day
            model.addConstr(gb.quicksum(meetings[(t, d, h)] for h in range(num_cols)) <= 4)

            # Constraint 2: Each participant attends at most one meeting per time slot and day
            for p in persons:
                model.addConstr(gb.quicksum(meetings[(t, d, h)] for h in range(num_cols)
                                      if cols[d][t][h][p] == 1) <= 1)
                
            # Constraint 3: No back-to-back meetings for participants
            if t != 34:
                for h in range(len(cols[d][t])):   
                    for h2 in range(len(cols[d][t+1])): # this is crucial as h varies!!!
                        for p in persons:
                            #print(t, d, h, p)
                            if cols[d][t][h][p] == 1 and cols[d][t+1][h2][p] == 1:
                                model.addConstr(meetings[(t, d, h)] + meetings[(t+1, d, h2)] <= 1)


    # Objective: Maximize number of meetings
    model.setObjective(gb.quicksum(meetings[(t, d, h)] for t in times
                                    for d in days for h in range(len(cols[d][t]))), gb.GRB.MAXIMIZE)

    print('solving model...')
    
    # Solve the optimization problem
    model.optimize()

    # Print solution
    if model.status == gb.GRB.OPTIMAL:
        with open(f'{filename}.tsv', 'w') as sol:
            sol.truncate(0)
            sol.write('D\tT\ta*\n')
            print("Optimal solution found!")
            for d in days:
                for t in times:
                    for h in range(len(cols[d][t])):
                        if meetings[(t, d, h)].x > 0.5:
                            sol.write(f"{d}\t{t}\t{cols[d][t][h]}\n")
    else:
        print("No solution found.")
    return model


In [9]:
m = maximize_meetings(data[data['h'] <= 500],'sol_lt_500')

solving model...
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50434522 rows, 77809 columns and 101623036 nonzeros
Model fingerprint: 0x4228ca4d
Variable types: 0 continuous, 77809 integer (77809 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 89.0000000
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve removed 3974 rows and 0 columns (presolve time = 12s) ...
Presolve removed 3974 rows and 0 columns (presolve time = 18s) ...
Presolve removed 3974 rows and 0 columns (presolve time = 22s) ...
Presolve removed 3974 rows and 0 columns (presolve time = 28s) ...
Presolve removed 3974 rows and 0 columns (presolve time = 30s) ...
Presolve removed 3974 ro

In [ ]:
m = m = maximize_meetings(data[data['h'] <= 1000],'sol_lt_1k')